In [14]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import preprocessing
import seaborn as sns
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

In [2]:
df = pd.read_csv("data/temp_merged.csv")

In [3]:
df = df.drop(columns=['Unnamed: 0', 'FlightDate', 'Quarter', 'Year', 'Month', 'DayofMonth',
       'Origin', 'Dest','ArrTime'])
y = df['ArrDel15']
X = df.drop(columns=['ArrDel15','ArrDelayMinutes'])

In [4]:
#FEATURE SET
X.columns

Index(['DepTime', 'DepDel15', 'CRSDepTime', 'DepDelayMinutes',
       'OriginAirportID', 'DestAirportID', 'CRSArrTime', 'windspeedKmph_x',
       'DewPointF_x', 'cloudcover_x', 'precipMM_x', 'pressure_x',
       'WindGustKmph_x', 'visibility_x', 'weatherCode_x', 'tempF_x',
       'WindChillF_x', 'winddirDegree_x', 'humidity_x', 'windspeedKmph_y',
       'DewPointF_y', 'cloudcover_y', 'precipMM_y', 'pressure_y',
       'WindGustKmph_y', 'visibility_y', 'weatherCode_y', 'tempF_y',
       'WindChillF_y', 'winddirDegree_y', 'humidity_y'],
      dtype='object')

## SMOTE OVERSAMPLING

In [44]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.7)
sm = RandomOverSampler()
x_train, y_train = sm.fit_sample(x_train, y_train)
sampled_df = pd.concat([pd.DataFrame(x_train), pd.DataFrame(y_train)], axis=1)

k=X.columns
z=pd.DataFrame(y,columns=['ArrDel15'])
sampled_df.columns=k.append(z.columns)

In [ ]:
sampled_df.ArrDel15.value_counts()

#### Using selectKBest to select best features

In [112]:
"""
min_max_scaler = preprocessing.MinMaxScaler()
Scaled_X = min_max_scaler.fit_transform(x_train)

bestfeatures = SelectKBest(chi2, k=15)
fit = bestfeatures.fit(Scaled_X,y_train)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  
feats = list(featureScores.nlargest(22,'Score').Specs)
x_train = sampled_df[feats]
x_test = x_test[feats]

"""

# LOGISTIC REGRESSION

In [45]:
lr = linear_model.LogisticRegression(max_iter=5000)
clf = lr.fit(x_train, y_train)

In [46]:
print("Logistic regression Train Accuracy : ", clf.score(x_train,y_train))
print("Logistic regression Test Accuracy  : ", metrics.accuracy_score(y_test, lr.predict(x_test)))

print(confusion_matrix(y_test,lr.predict(x_test),labels=[1,0]))

print(classification_report(y_test,lr.predict(x_test)))

Logistic regression Train Accuracy :  0.8543605145018521
Logistic regression Test Accuracy  :  0.8961147219271556
[[ 90438  25831]
 [ 31870 407291]]
              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93    439161
         1.0       0.74      0.78      0.76    116269

    accuracy                           0.90    555430
   macro avg       0.84      0.85      0.85    555430
weighted avg       0.90      0.90      0.90    555430



# RANDOM FOREST CLASSIFIER

In [47]:
rf = RandomForestClassifier(criterion='entropy')
clf = rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)

In [48]:
print(" Train Accuracy : ", clf.score(x_train, y_train))
print(" Test Accuracy  : ", metrics.accuracy_score(y_test, y_pred))

print("CONFUSION MATRIX\n",confusion_matrix(y_test, y_pred, labels=[1,0]))

print(classification_report(y_test, y_pred))

 Train Accuracy :  0.9999985354623155
 Test Accuracy  :  0.9153376663125866
CONFUSION MATRIX
 [[ 85368  30901]
 [ 16123 423038]]
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.95    439161
         1.0       0.84      0.73      0.78    116269

    accuracy                           0.92    555430
   macro avg       0.89      0.85      0.87    555430
weighted avg       0.91      0.92      0.91    555430



# EXTRA TREES CLASSIFIER

In [49]:
ex = ExtraTreesClassifier(criterion='entropy')
clf = ex.fit(x_train,y_train)
y_pred = ex.predict(x_test)

In [50]:
print(" Train Accuracy : ", clf.score(x_train,y_train))
print(" Test Accuracy  : ", metrics.accuracy_score(y_test, y_pred))

print("CONFUSION MATRIX\n",confusion_matrix(y_test,y_pred,labels=[1,0]))

print(classification_report(y_test,y_pred))

 Train Accuracy :  0.9999995118207718
 Test Accuracy  :  0.911209333309328
CONFUSION MATRIX
 [[ 85914  30355]
 [ 18962 420199]]
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.94    439161
         1.0       0.82      0.74      0.78    116269

    accuracy                           0.91    555430
   macro avg       0.88      0.85      0.86    555430
weighted avg       0.91      0.91      0.91    555430



#  XGBOOST CLASSIFIER

In [51]:
xg = XGBClassifier()
clf = xg.fit(x_train, y_train)
y_pred = xg.predict(x_test)

In [52]:
print(" Train Accuracy : ", clf.score(x_train, y_train))
print(" Test Accuracy  : ", metrics.accuracy_score(y_test, y_pred))

print("CONFUSION MATRIX\n",confusion_matrix(y_test, y_pred, labels=[1,0]))

print(classification_report(y_test, y_pred))

 Train Accuracy :  0.8684786577805029
 Test Accuracy  :  0.8964045874367607
CONFUSION MATRIX
 [[ 93247  23022]
 [ 34518 404643]]
              precision    recall  f1-score   support

         0.0       0.95      0.92      0.93    439161
         1.0       0.73      0.80      0.76    116269

    accuracy                           0.90    555430
   macro avg       0.84      0.86      0.85    555430
weighted avg       0.90      0.90      0.90    555430



## DECISON TREES CLASSIFIER

In [53]:
ds = DecisionTreeClassifier()
clf = ds.fit(x_train, y_train)
y_pred = ds.predict(x_test)

In [54]:
print(" Train Accuracy : ", clf.score(x_train, y_train))
print(" Test Accuracy  : ", metrics.accuracy_score(y_test, y_pred))

print("CONFUSION MATRIX\n",confusion_matrix(y_test, y_pred, labels=[1,0]))

print(classification_report(y_test, y_pred))

 Train Accuracy :  0.9999995118207718
 Test Accuracy  :  0.8718236321408638
CONFUSION MATRIX
 [[ 80994  35275]
 [ 35918 403243]]
              precision    recall  f1-score   support

         0.0       0.92      0.92      0.92    439161
         1.0       0.69      0.70      0.69    116269

    accuracy                           0.87    555430
   macro avg       0.81      0.81      0.81    555430
weighted avg       0.87      0.87      0.87    555430

